In [1]:
from copy import deepcopy
import nltk
import numpy as np
import pynlpl.lm.lm as lm
import MultiTree as MT
from time import time
from decoding import get_best_paths, power, times, divide

ImportError: No module named pynlpl.lm.lm

In [2]:
%load_ext line_profiler

In [3]:
measures = np.load("phon_measures.npy")
measures.shape

(40,)

In [4]:
model = lm.ARPALanguageModel('lang_model/3-gram-3.ARPA', base_e = False)

In [5]:
model.scoreword("the", history = ("hello","i", "am"))

-2.647338

In [6]:
def pr_lang(x, y):
    return 10.**(model.scoreword(x, history = tuple(y.word_hist)))

In [7]:
phon_probs_ = np.load("phoneme_probs.npy")
phon_probs_.shape

# for i in range(phon_probs_.shape[0]):
#     phon_probs_[i][39] /= 9.
#     row_sum = np.sum(phon_probs_[i])
#     phon_probs_[i] /= row_sum
phon_probs = phon_probs_[30:]

In [8]:
from process_data_temp import interpret_probs
print "".join(interpret_probs(phon_probs))

Using TensorFlow backend.


-W-AH-T-IH-Z-DHAH-S-TH-OW-D-L-AW-M-B-AY-


In [9]:
phon_dict = {MT.phonemes[k] : k for k in range(39)}
phon_dict["-"] = 39

In [10]:
def pr_ctc(phon, t):
    phon_idx = phon_dict[phon]
    return phon_probs[t-1][phon_idx] #/ measures[phon_idx]

In [11]:

#removes stresses from vowels

phons = [u'AA',u'AE',u'AH',u'AO',u'AW',u'AY',u'B',u'CH',
u'D',u'DH',u'EH',u'ER',u'EY',u'F',u'G',u'HH',u'IH',u'IY',u'JH',
u'K',u'L',u'M',u'N',u'NG',u'OW',u'OY',u'P',u'R',u'S',u'SH',u'T',
u'TH',u'UH', u'UW',u'V',u'W',u'Y',u'Z',u'ZH', u'-']

phoneme_dict = dict(zip(phons, range(40)))

MAX_WORDS = 16000
S = np.load("unigram_scores.npz")["arr_0"]
print "First 10 items in S are:"
print S[:10]
print "Last 10 items in S are:"
print S[MAX_WORDS-10:MAX_WORDS]
D = nltk.corpus.cmudict.dict()
print "Original dictionary has {} words".format(len(D))
D_small = {k: D[k] for k in S[:MAX_WORDS,0]}
print "Small dictionary has {} words".format(len(D_small))


def strip_stresses(phoneme):
    main_phon = phoneme[:-1]
    last_letter = phoneme[-1]
    if last_letter.isnumeric():
        return main_phon
    else:
        return phoneme
    
def obtain_phonemes(sentence, dictionary):
    sentence_phonemes = []
    for word in sentence:
        word = word.lower()
        if word in dictionary:
            #nltk.corpus.cmudict.dict() gives a list of lists, giving both
            #the american and english pronounciations
            phonemes = dictionary[word][0]
            stripped_phonemes = map(strip_stresses, phonemes)
            sentence_phonemes.extend(stripped_phonemes)
        else:
            #print "{} is not in the dictionary, can't get phonemes!".format(word)
            return None
    return sentence_phonemes

R = {}
# assert len(D.items()) ==len(set([" ".join([strip_stresses(phon) for phon in h]) for homophones in D.values() for h in homophones]))
for word, hs in D_small.items():
    for h in hs:
        stripped = [strip_stresses(phon) for phon in h]
        phon_string = " ".join(stripped)
        if R.get(phon_string):
            R[phon_string].append(word)
        else:
            R[phon_string] = [deepcopy(word)]
# for a, b in R.items():
#     if a not in [" ".join([strip_stresses(p) for p in D[x][0]]) for x in ["what", "is", "the", "phone", "number"] ]:
#         del R[a]

print "R has {} keys".format(len(R.keys()))
print ""
Rkeys = R.keys()

First 10 items in S are:
[[u'the' u'7.615819']
 [u'to' u'7.263301']
 [u'of' u'7.242066']
 [u'a' u'7.222159']
 [u'and' u'7.210459']
 [u'in' u'7.173804']
 [u'that' u'6.843069']
 [u"'s" u'6.838454']
 [u'for' u'6.83241']
 [u'on' u'6.775705']]
Last 10 items in S are:
[[u'blindness' u'3.281488']
 [u'devout' u'3.281488']
 [u'feasibility' u'3.281488']
 [u'succumbed' u'3.281488']
 [u'fitzpatrick' u'3.281488']
 [u'kidman' u'3.281261']
 [u'lenses' u'3.281261']
 [u'extremes' u'3.281261']
 [u'portrays' u'3.281033']
 [u'parkway' u'3.281033']]
Original dictionary has 123455 words
Small dictionary has 16000 words
R has 18433 keys



In [12]:
D["tuberculosis"]

[[u'T',
  u'AH0',
  u'B',
  u'ER2',
  u'K',
  u'Y',
  u'AH0',
  u'L',
  u'OW1',
  u'S',
  u'IH0',
  u'S'],
 [u'T',
  u'UW0',
  u'B',
  u'ER2',
  u'K',
  u'Y',
  u'AH0',
  u'L',
  u'OW1',
  u'S',
  u'AH0',
  u'S'],
 [u'T',
  u'UW0',
  u'B',
  u'ER2',
  u'K',
  u'Y',
  u'UW0',
  u'L',
  u'OW1',
  u'S',
  u'AH0',
  u'S']]

In [23]:
ext_store = {}
n_done = 0
for spaced_phons, homs in deepcopy(R.items()):
    if homs == ["T UW M ER Z"]:
        print spaced_phons
    phon_list = spaced_phons.split(" ")
    for i in range(len(phon_list)):
        sublist = phon_list[:i]
        key = " ".join(sublist)
        if key in ext_store.keys():
            for hom in homs:
                if not hom in ext_store[key]:
                    ext_store[key].append(hom)
        else:
            ext_store[key] = deepcopy(homs)
    n_done +=1
    if n_done % 1000 == 0:
        print n_done
            

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [24]:
ext_store["T UW"]

[u'tomorrow',
 u'tumor',
 u'tuna',
 u'tumors',
 u'tucson',
 u'tuned',
 u'tunes',
 u'tuesday',
 u'tuberculosis',
 u'tubes',
 u'tumultuous',
 u'tooth',
 u'toulouse',
 u'tube',
 u'tune',
 u'tool',
 u'tomb',
 u'tools',
 u'tonight',
 u'today',
 u'tunisia']

In [27]:
D_small["tuberculosis"]

[[u'T',
  u'AH0',
  u'B',
  u'ER2',
  u'K',
  u'Y',
  u'AH0',
  u'L',
  u'OW1',
  u'S',
  u'IH0',
  u'S'],
 [u'T',
  u'UW0',
  u'B',
  u'ER2',
  u'K',
  u'Y',
  u'AH0',
  u'L',
  u'OW1',
  u'S',
  u'AH0',
  u'S'],
 [u'T',
  u'UW0',
  u'B',
  u'ER2',
  u'K',
  u'Y',
  u'UW0',
  u'L',
  u'OW1',
  u'S',
  u'AH0',
  u'S']]

In [28]:
from decoding import DecodeInfo

In [29]:
T = len(phon_probs)
T

89

In [30]:
pr_nb_store = [{} for i in range(T)]
pr_b_store = [{} for i in range(T)]

p_ctc = pr_ctc
p_lang = pr_lang
def pr_nb(t, y):
        if y in pr_nb_store[t]:
            return pr_nb_store[t][y]
        else:
            return 0.

def pr_b(t, y):
    if y in pr_b_store[t]:
        return pr_b_store[t][y]
    else:
        return 0.

def pr_nb_add(t, y, val):
    if y in pr_nb_store[t]:
        pr_nb_store[t][y] += val
    else:
        pr_nb_store[t][y] = val

def pr_b_add(t, y, val):
    if y in pr_b_store[t]:
        pr_b_store[t][y] += val
    else:
        pr_b_store[t][y] = val

#extension probability of adding a phoneme k
# CHANGED THIS
def p_ext(k, y, t):
    if y.last_phoneme() == k:
        p_last = pr_b(t-1, y)
    else:
        p_last = pr_nb(t-1, y) + pr_b(t-1, y)            
    
    a = p_ctc(k,t)
    b = p_trans(k, y)
    bp = power(b,beta)
    
    return times([a, bp, p_last])

#derived phoneme transition prob from language model
def p_trans(k, y):
    if y.tree.lookup([k]) == None:
        return 0.
    else:
        denom_leaves = ext_store.get(" ".join(y.partial_word), [])
        numer_leaves = ext_store.get(" ".join(y.partial_word) + " " + k, [])
        denom_ps = [p_lang(x, y) for x in denom_leaves]
        numer_ps = [p_lang(x, y) for x in numer_leaves]
        denom = np.sum(np.asarray(denom_ps))
        numer = np.sum(np.asarray(numer_ps))
        return divide(numer, denom)

#penalty probability for converting to a homophone
def p_homophone(word, y):
    denom = np.sum(np.asarray([p_lang(x, y) for x in ext_store.get(" ".join(y.partial_word), [])]))
    numer = p_lang(word, y)
    return divide(numer, denom)
    

def test_beam_search(T, p_ctc, p_lang, width, beta):
    
    #initialising non-blank prob
    empty_str = DecodeInfo()
    pr_nb_store[0][empty_str] = 1.
    pr_b_store[0][empty_str] = 0.
    
    #most probable sequences
    B = [empty_str]
    
    for t in range(1, T+1):
            B_hat = get_best_paths(B, pr_b, pr_nb, width, t-1)
            print "\n".join(str(elem) + " : {}".format(pr_b(t-1, elem) + pr_nb(t-1, elem)) for elem in B_hat) + "\n\n"
            B = []
            for y in B_hat:
                if not y.empty():
                    pr_nb_store[t][y]= times([pr_nb(t-1, y), p_ctc(y.last_phoneme(), t)])
                    
                    #dont think we should have this - paths which decode to the same thing are accounted for in the
                    #loop below
#                     if y.one_fewer() in B_hat:
#                         pr_nb_store[t][y] += p_ext(y.last_phoneme(), y.one_fewer(), t)

                pr_b_store[t][y] = times(pr_b(t-1,y) + pr_nb(t-1, y), p_ctc("-", t))
                
                B.append(y)
                
                for k in MT.phonemes:

                    #if adding a phoneme k is valid
                    if k in y.tree.branches:

                        pr_plus = p_ext(k, y, t)

                        #initialising y_new = y+k
                        y_new = DecodeInfo(orig = y)
                        y_new.add_phon(k)

                        conversion_penalty_prob = 0.
                        #handles conversion from phoneme string to word
                        for word in y_new.tree.leaf:

                            #converted_y converts phoneme part of y_new into a new word
                            converted_y = DecodeInfo()
                            converted_y.word_hist = deepcopy(y_new.word_hist)
                            converted_y.word_hist.append(word)
                            pr_b_add(t, converted_y, 0.)
                            pr_nb_add(t, converted_y, times(pr_plus, power(p_homophone(word, y_new), beta)))
                            B.append(converted_y)
                            conversion_penalty_prob += p_homophone(word, y_new)

                        #penalises non-converted string
                        pr_nb_add(t, y_new, times(pr_plus, power(1.0 - conversion_penalty_prob, beta)))
                        B.append(y_new)

    return (B, pr_b_store, pr_nb_store)#get_best_paths(B, pr_b_store, pr_nb_store, 1)

In [31]:
beta = 0.05

In [35]:
%lprun -f p_lang test_beam_search(T, pr_ctc, pr_lang, 10, beta)

| : 1.0


| : 0.995358049646
o'| : 0.0
|ER : 0.0
|TH : 0.0
ou| : 0.0
i| : 0.0
|EY : 0.0
|AW : 0.0
eaux| : 0.0
|S : 0.0


| : 0.991759808681
i|V : 0.0
i|W : 0.0
i|T : 0.0
i|R : 0.0
i|S : 0.0
i|P : 0.0
i|Z : 0.0
i|Y : 0.0
i|F : 0.0


| : 0.808761114311
i|V : 0
i|W : 0
i|T : 0
i xie| : 0.0
i|R : 0
i|S : 0
i|P : 0
i suh| : 0.0
i roi| : 0.0


| : 0.0558780578637
i|V : 0
i|W : 0
i|T : 0
i xie| : 0
i|R : 0
i|S : 0
i|P : 0
i suh| : 0
i roi| : 0


| : 0.00211543263787
i|V : 0
i|W : 0
i|T : 0
i xie| : 0
i|R : 0
i|S : 0
i|P : 0
i suh| : 0
i roi| : 0


| : 0.00206998779851
i|V : 0
i|W : 0
i|T : 0
i xie| : 0
i|R : 0
i|S : 0
i|P : 0
i suh| : 0
i roi| : 0


*** KeyboardInterrupt exception caught in code being profiled.

In [73]:
pr_b_store[2][DecodeInfo()]

1.8625862530573387

In [63]:
pr_nb_store = [{}]*(T+1)
pr_b_store = [{}]*(T+1)


In [74]:
DecodeInfo().empty()

True

In [21]:
power(-1,0)

nan